In [ ]:
### Calling my latest (last months) equity ticker list from yahoo_fin, retrieving dividends and putting into SQLite DB 
### Going to new table in WSB SQLite
### Last Updated 2022_0620
### 

In [4]:
import yfinance as yf
import json
import requests # get connection
import pandas as pd
import json
from sqlalchemy import create_engine
import sqlite3 as db
import schedule
import time
import apscheduler
import apscheduler.schedulers.background
import yahoo_fin.stock_info as si
from yahoo_fin.stock_info import get_analysts_info

In [12]:
### Pull my current tickers in from sorted sheet
# read 2nd sheet of an excel file
bp_tickers = pd.read_excel(r'O:\_Tiller on One Drive\Equity Dividend Tracker\tickers_for_yfinance.xlsx', sheet_name = "current_unique_bp_tickers")
#bp_tickers.head()

,symbol,date
0,AAP,2022-05-31
1,AAPL,2022-05-31
2,ABBV,2022-05-31
3,ABR,2022-05-31
4,ACC,2022-05-31


In [6]:
### Make my tickers df sybols column into a list
# Converting DataFrame to a list containing
# all the rows of column 'Name'
tickers = bp_tickers['symbol'].tolist()
bp_dow_list = tickers
 
# Printing the converted list.
#print(bp_dow_list)

['AAP', 'AAPL', 'ABBV', 'ABR', 'ACC', 'ADC', 'ADM', 'ADP', 'AEP', 'AFL', 'AGNC', 'AIRC', 'ALB', 'ALCO', 'ALL', 'ALLY', 'AM', 'AMAT', 'AMGN', 'AMH', 'AMT', 'AMZA', 'AMZN', 'AOS', 'APD', 'APOG', 'AQN', 'AR', 'ARES', 'ATO', 'AVA', 'AVB', 'AVGO', 'AWK', 'AWR', 'AXP', 'AZN', 'BAC', 'BAX', 'BBUC', 'BCI', 'BHP', 'BIP', 'BIZD', 'BK', 'BKH', 'BLK', 'BMO', 'BMY', 'BNS', 'BP', 'BRG', 'BRKB', 'BXP', 'C', 'C061579', 'CAT', 'CBOE', 'CCI', 'CERN', 'CFG', 'CGW', 'CHRW', 'CL', 'CLPR', 'CLX', 'CME', 'CMI', 'CNA', 'CNI', 'COST', 'CPB', 'CPT', 'CRM', 'CRWD', 'CSCO', 'CSL', 'CTRE', 'CUBE', 'CVX', 'CWCO', 'CWENA', 'D', 'DBC', 'DEA', 'DFAC', 'DFNM', 'DFS', 'DGRO', 'DGRW', 'DHS', 'DIV', 'DJD', 'DLR', 'DOC', 'DOV', 'DOW', 'DPZ', 'DRE', 'DRI', 'DSU', 'DUK', 'DVN', 'ED', 'EDV', 'EFC', 'EIX', 'ELS', 'EMLC', 'EMR', 'EMXC', 'ENB', 'EPD', 'EQIX', 'ERIE', 'ERTH', 'ESGD', 'ESGE', 'ESGU', 'ESGV', 'ET', 'ETN', 'ETR', 'EWI', 'EXR', 'FAST', 'FB', 'FBC', 'FCX', 'FDX', 'FE', 'FHLC', 'FITB', 'FLNG', 'FMAG', 'FNF', 'FPI', 'FR

In [7]:
### Retrieve dividend history for xxxx date to present
dow_stats = {}
for ticker in tickers:
    temp = si.get_dividends(ticker, "01-01-2020")
    dow_stats[ticker] = temp
#dow_stats

{'AAP':             dividend ticker
 2020-03-19      0.25    AAP
 2020-06-18      0.25    AAP
 2020-09-17      0.25    AAP
 2020-12-17      0.25    AAP
 2021-03-18      0.25    AAP
 2021-06-17      1.00    AAP
 2021-09-16      1.00    AAP
 2021-12-16      1.00    AAP
 2022-03-17      1.50    AAP
 2022-06-16      1.50    AAP,
 'AAPL':             dividend ticker
 2020-02-07    0.1925   AAPL
 2020-05-08    0.2050   AAPL
 2020-08-07    0.2050   AAPL
 2020-11-06    0.2050   AAPL
 2021-02-05    0.2050   AAPL
 2021-05-07    0.2200   AAPL,
 'ABBV':             dividend ticker
 2020-01-14      1.18   ABBV
 2020-04-14      1.18   ABBV
 2020-07-14      1.18   ABBV
 2020-10-14      1.18   ABBV
 2021-01-14      1.30   ABBV
 2021-04-14      1.30   ABBV
 2021-07-14      1.30   ABBV
 2021-10-14      1.30   ABBV
 2022-01-13      1.41   ABBV
 2022-04-13      1.41   ABBV,
 'ABR':             dividend ticker
 2020-02-27      0.30    ABR
 2020-06-29      0.30    ABR
 2020-08-14      0.31    ABR
 2020-11-1

In [8]:
### combine all into a single data frame and reset the index 
bp_ticker_div_pd = pd.concat(dow_stats)
bp_ticker_div_pd = bp_ticker_div_pd.reset_index()
bp_ticker_div_pd

,level_0,level_1,dividend,ticker
0,AAP,2020-03-19,0.250,AAP
1,AAP,2020-06-18,0.250,AAP
2,AAP,2020-09-17,0.250,AAP
3,AAP,2020-12-17,0.250,AAP
4,AAP,2021-03-18,0.250,AAP
...,...,...,...,...
4320,ZTS,2021-01-19,0.250,ZTS
4321,ZTS,2021-04-20,0.250,ZTS
4322,ZTS,2021-07-20,0.250,ZTS
4323,ZTS,2021-10-28,0.250,ZTS


In [11]:
### To SQLite on local drive

# Creates and/or connects to SQLite DB Named lotto_results1.db
connection = db.connect(r'P:\Luke Barosso Python\db_house\wsb_df.db')

#Appends the above DF to the DB Created into a table named mega_millions1_df
bp_ticker_div_pd.to_sql(name="bp_ticker_div_df", con=connection, if_exists='append', index=False)

## Commit connection
connection.commit()

In [9]:
### Check output in pd format
#bp_ticker_div_pd.head()

,level_0,level_1,dividend,ticker
0,AAP,2020-03-19,0.25,AAP
1,AAP,2020-06-18,0.25,AAP
2,AAP,2020-09-17,0.25,AAP
3,AAP,2020-12-17,0.25,AAP
4,AAP,2021-03-18,0.25,AAP


In [7]:
### save into a csv 
#bp_ticker_div_pd.to_csv(r'O:\_Tiller on One Drive\Equity Dividend Tracker\bp_ticker_div_pd.csv')